<a href="https://colab.research.google.com/github/palswayam5/AI_generated_Images_Classifier/blob/master/ai_generated_image_res_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -U keras-tuner

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Input,Activation,BatchNormalization,MaxPooling2D,AveragePooling2D,ZeroPadding2D,Add,Dropout
from tensorflow.keras import Input,Model
from tensorflow.keras.initializers import GlorotUniform as glorot_uniform
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import keras_tuner as kt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_shape = (20,20,3)
classes = 2

In [ ]:
def identity_block(X, f, filters, stage, block,l):
    """
    Implementation of the identity block

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0), kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)


    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2,l = 0.01):
    """
    Implementation of the convolutional block

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    # Second component of main path
    X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu') (X)

    # Third component of main path
    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)


    return X

In [ ]:
def ResNet50(hp):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1,l = 0)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b',l=0)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c',l=0)


    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b',l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c',l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d',l = 0)

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f',l = 0)

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b',l = 0)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c',l = 0)

    # AVGPOOL . Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D()(X)

    # output layer
    X = Flatten()(X)
    hp_units = hp.Int('units', min_value=125, max_value=512, step=32)
    #X = Dense(units = 224,activation = 'relu')(X)
    X = Dense(units = hp_units,activation = 'relu')(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=0.01))(X)


    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate  = hp_learning_rate),loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

    return model

In [ ]:
#model = ResNet50()
tuner = kt.Hyperband(ResNet50,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt1')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')

In [ ]:
X = df.drop('labels',axis = 1).to_numpy()
y = df['labels'].to_numpy()
X = X.reshape(-1,20,20,3)
y = y.reshape(-1,1)
print(X.shape,y.shape)

(5250, 20, 20, 3) (5250, 1)


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X, y, epochs=50, validation_split=0.4, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X, y, epochs=50, validation_split=0.4)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
99/99 [==============================] - 62s 68ms/step - loss: 1.1491 - accuracy: 0.6968 - val_loss: 0.6455 - val_accuracy: 0.7252
Epoch 2/50
99/99 [==============================] - 5s 55ms/step - loss: 0.6248 - accuracy: 0.7324 - val_loss: 0.6219 - val_accuracy: 0.7252
Epoch 3/50
99/99 [==============================] - 4s 43ms/step - loss: 0.5780 - accuracy: 0.7365 - val_loss: 0.6461 - val_accuracy: 0.7252
Epoch 4/50
99/99 [==============================] - 4s 43ms/step - loss: 0.5283 - accuracy: 0.7479 - val_loss: 0.5479 - val_accuracy: 0.7514
Epoch 5/50
99/99 [==============================] - 6s 58ms/step - loss: 0.4783 - accuracy: 0.7870 - val_loss: 0.5383 - val_accuracy: 0.7648
Epoch 6/50
99/99 [==============================] - 4s 44ms/step - loss: 0.3866 - accuracy: 0.8340 - val_loss: 0.4271 - val_accuracy: 0.8314
Epoch 7/50
99/99 [==============================] - 4s 44ms/step - loss: 0.3105 - accuracy: 0.8784 - val_loss: 0.5117 - val_accuracy: 0.8214
Epoch 8/50
9

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X,y, epochs=best_epoch, validation_split=0.4)

Epoch 1/34
99/99 [==============================] - 44s 61ms/step - loss: 1.1273 - accuracy: 0.7079 - val_loss: 0.6342 - val_accuracy: 0.7252
Epoch 2/34
99/99 [==============================] - 4s 44ms/step - loss: 0.6274 - accuracy: 0.7356 - val_loss: 0.6261 - val_accuracy: 0.7252
Epoch 3/34
99/99 [==============================] - 5s 53ms/step - loss: 0.5860 - accuracy: 0.7387 - val_loss: 0.6096 - val_accuracy: 0.7252
Epoch 4/34
99/99 [==============================] - 4s 44ms/step - loss: 0.5394 - accuracy: 0.7397 - val_loss: 0.5709 - val_accuracy: 0.7252
Epoch 5/34
99/99 [==============================] - 4s 45ms/step - loss: 0.4827 - accuracy: 0.7841 - val_loss: 0.4214 - val_accuracy: 0.8224
Epoch 6/34
99/99 [==============================] - 5s 53ms/step - loss: 0.3889 - accuracy: 0.8416 - val_loss: 0.4555 - val_accuracy: 0.8381
Epoch 7/34
99/99 [==============================] - 4s 44ms/step - loss: 0.2864 - accuracy: 0.8905 - val_loss: 0.5133 - val_accuracy: 0.8190
Epoch 8/34
9

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.4)

In [ ]:
y_pred_val = hypermodel.predict(X_val)
y_pred_final = np.zeros([y_pred_val.shape[0],1])
for i in range(y_pred_val.shape[0]):
  if(y_pred_val[i][0]>y_pred_val[i][1]):
    y_pred_final[i] = 0
  else:
    y_pred_final[i] = 1

66/66 [==============================] - 1s 9ms/step


In [ ]:
y_pred_val

array([[9.8469430e-01, 1.5305687e-02],
       [9.9996901e-01, 3.0972333e-05],
       [1.0825742e-02, 9.8917425e-01],
       ...,
       [9.9993873e-01, 6.1311970e-05],
       [9.8412554e-04, 9.9901581e-01],
       [9.9742532e-01, 2.5746527e-03]], dtype=float32)

In [ ]:
f1_score(y_val,y_pred_final)

0.9233543733092876

In [ ]:
df_test.head()

,id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,1,-3.388242,0.868285,-0.427619,-0.678964,-1.625735,0.262761,1.243040,1.537751,-0.352028,...,-0.776403,-0.662884,-0.257091,-1.168413,0.223260,-0.482520,-0.085453,-0.382265,-0.539349,-1.682404
1,2,-0.496920,0.952381,0.989040,0.451422,0.513516,-0.099658,-1.124326,0.729430,-0.216224,...,0.379635,-1.760084,1.125450,-0.328047,-0.880305,-1.257607,0.964312,2.021104,0.655021,-0.423029
2,3,1.128369,-0.537951,2.544358,1.165254,-1.904994,0.776961,-0.495768,0.060111,-1.418468,...,1.165254,-1.373589,-0.483701,-0.964782,-0.869555,0.066040,-0.444567,-0.531935,-0.878660,1.099488
3,4,0.051253,1.746814,0.681177,1.844524,-0.327977,1.226839,-0.085519,0.379008,-1.003667,...,-0.442288,-2.794472,-0.763468,-0.789832,-0.113209,-2.703150,-2.058728,1.070627,-0.458045,-0.435825
4,5,1.423209,-0.983594,-1.694170,1.197507,1.044211,0.518777,-0.298612,-0.365174,0.738447,...,-2.624450,-3.200223,0.711422,-0.190394,0.337224,-1.656639,0.707360,-0.562290,1.471181,-0.192000


In [ ]:
X_test = df_test.drop('id',axis = 1).to_numpy()
id = df_test['id'].to_numpy()
X_test = X_test.reshape(-1,20,20,3)
id = id.reshape(-1,1)

In [ ]:
y_test = hypermodel.predict(X_test)
y_test_final = np.zeros([y_test.shape[0],1])
for i in range(y_test.shape[0]):
  if(y_test[i][0]>y_test[i][1]):
    y_test_final[i] = 0
  else:
    y_test_final[i] = 1

71/71 [==============================] - 1s 11ms/step


In [ ]:
ans = np.concatenate([id,y_test_final],axis = 1)

In [ ]:
submission = pd.DataFrame(ans,columns = ['id','labels'])

In [ ]:
submission.to_csv('res50.csv')